In [2]:
import socket
import threading
import pandas as pd
import os
import hashlib

In [3]:
df=pd.read_csv('DataBase.csv')


In [4]:
df

,Username,Password,Number
0,x,1c89d7180a80a614bf0cfa014e613be0ffd92867,5
1,z,1c89d7180a80a614bf0cfa014e613be0ffd92867,5
2,lm,c6f32f45655c68f09551414523bbe4b899678fde,5
3,ss,fdd14431ceeaf9a48c3810265c3d10ddef5e3990,4


In [10]:

df=pd.read_csv('DataBase.csv')
#hash func
def hashSHA1(password):
    if isinstance(password, str):
        password = password.encode('utf-8') 
    hash_sha1 = hashlib.sha1()
    hash_sha1.update(password)
    result = hash_sha1.hexdigest()  
    return result


def CheckPassword(username, password):
    print("Checking password")
    #hash one more time so we had n times hash of password / input=bytes
    password2 = bytes.fromhex(password)   
    hashpassword = hashSHA1(password2)
    ans = df[(df['Username'] == username) & (df['Password'] == hashpassword)]
    #username and pass found
    if not ans.empty:
        i = ans.index[0]
        df.at[i, 'Number'] -= 1 
        df['Password'][i]=password 
        df.to_csv('DataBase.csv', index=False)  
        return 100
    else:
        return -100

def SaveData(username,password,n):
    if new:
        #n=10
        if df[df['Username'] == username].empty:
            df.loc[len(df)]= [username,password,n]
            df.to_csv('DataBase.csv',index=False)
            return 100
        else:
            return -100
        
    else:
        df.loc[len(df)]= [username,password,n-1]
        df.to_csv('DataBase.csv',index=False)
        return 100

def GetUserpass(msg):
    if msg.startswith('UserNumber'):
        return msg[len('UserNumber'):]
    if msg.startswith('UserName'):
        return msg[len('UserName'):]
    if msg.startswith('Password'):
        return msg[len('Password'):]
    
def getN(username):
    n=df.loc[df['Username']==username, 'Number'].values
    return n[0] if len(n) > 0 else -1

def multi_client(new,connection, client_addr,n):
    global df
    print(f"Connected to{client_addr}")
    try:
    
        while(True):
            data=connection.recv(1024)
            if data:
                msg=data.decode()
                msg=str(msg)
                if msg.startswith('UserName'):
                    username= GetUserpass(msg)
                    print(f"recieved from {client_addr}: {username}")
                    respons= "Username Recieved"

                #register
                elif msg.startswith('UserNumber'):
                    username= GetUserpass(msg)
                    new=False
                    print(f"recieved from {client_addr}: {username}")
                    n= str(getN(username))
                    respons=n

                elif msg.startswith('Password'):
                    password= GetUserpass(msg)
                    print(f"recieved from {client_addr}: {password}")
                    if new:
                        ans=SaveData(username,password,n)
                        if ans==100:
                            respons= "Set"
                        else:
                            respons="The username is already taken."
                    
                    else:
                        if CheckPassword(username,password)==100:
                            #n=1 client doesnt get hash
                            valid=int(n)-1
                            if valid>0:
                                valid=str(valid)
                                respons=f"Correct, Your password will be valide for {valid} more times"
                            else:
                                x=df[df['Username'] == username].index
                                df = df.drop(x)
                                df.to_csv('DataBase.csv', index=False)
                                respons=f"Correct, Your password is declined"
                        else:
                            respons='Your password or username is wrong.'

                connection.sendall(respons.encode())
                

            else:
                break
    except Exception as e:
        print(f"An error occurred with {client_addr}: {e}")
    finally:
        print(f"connection closed with {client_addr}")
                
        connection.close()

def start_server(new,n):
    server_socket=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
    addr=('localhost',65432)
    server_socket.bind(addr)
    server_socket.listen(5)
    print('Sever is waiting for a connections.')
    connection,client_addr=server_socket.accept()
    client_thread=threading.Thread(target=multi_client, args=(new,connection,client_addr,n))
    client_thread.start()
 
if __name__ =="__main__":
    new=True
    n=5
    start_server(new,n)


Sever is waiting for a connections.
Connected to('127.0.0.1', 58156)


recieved from ('127.0.0.1', 58156): kiss
recieved from ('127.0.0.1', 58156): b1c1d8736f20db3fb6c1c66bb1455ed43909f0d8
Checking password


C:\Users\ASUS\AppData\Local\Temp\ipykernel_22404\1867891110.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Password'][i]=password


connection closed with ('127.0.0.1', 58156)
